In [70]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [71]:
PINECONE_API_KEY = "pcsk_5ftJtk_RPGwskkYfaFGx21SBduyqwS44EGQ2stdCX3rJRdhB1s32dos2WfAJMH6hRDhq3B"

PINECONE_API_ENV = "us-east-1"

In [72]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [76]:
import pdfplumber

def load_pdf(pdf_path):
    """Extract text from a PDF using pdfplumber."""
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Extract text
extracted_data = load_pdf(r"C:\Users\mohda\OneDrive\Desktop\MEDICAL-CHATBOT\data\Medical_book.pdf")
print(extracted_data[:500])
 


GALE
The
ENCYCLOPEDIA
M
EDICINE
of
SECOND EDITION
G ALE
The
ENCYCLOPEDIA
M
EDICINE
of
SECOND EDITION
V O L U M E
1
A-B
JACQUELINE L. LONGE, EDITOR
DEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR
The GALE Since this page cannot legibly accommodate all copyright notices,the
acknowledgments constitute an extension of the copyright notice.
ENCYCLOPEDIA
While every effort has been made to ensure the reliability of the infor-
mation presented in this publication,the Gale Group neither guarantees
of MEDICINE


In [1]:
# extracted_data

In [78]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

In [79]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [80]:

# Split extracted text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_chunks = text_splitter.split_text(extracted_data)

print(f"Extracted {len(text_chunks)} text chunks from PDF.")


Extracted 5720 text chunks from PDF.


In [81]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [82]:

embeddings = download_hugging_face_embeddings()


In [83]:

from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Load the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
from langchain.embeddings import HuggingFaceEmbeddings

# Load the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
from langchain.embeddings import HuggingFaceEmbeddings

# 🔹 Load the Sentence Transformer model for embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 🔹 Convert a sample text into an embedding vector
text = "How does the heart function?"
query_embedding = embedding_model.embed_query(text)

print("Generated Embedding Vector:", query_embedding[:10])  # Print first 10 values


Generated Embedding Vector: [-0.004146425519138575, 0.05755056068301201, -0.001845396007411182, 0.014783775433897972, -0.04701027646660805, 0.01919114775955677, 0.003776268567889929, -0.07476205378770828, 0.04135068878531456, -0.024365302175283432]


In [84]:
embeddings = download_hugging_face_embeddings()


In [85]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [86]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [88]:
from pinecone import Pinecone

# 🔹 Step 1: Initialize Pinecone Client
pc = Pinecone(api_key="pcsk_5ftJtk_RPGwskkYfaFGx21SBduyqwS44EGQ2stdCX3rJRdhB1s32dos2WfAJMH6hRDhq3B")  # Replace with your API key

# 🔹 Step 2: Define Index Name and Region
index_name = "medicalchatbot"
region = "us-east-1"  # Change this to match your Pinecone region

# 🔹 Step 3: Check and Connect to Index
# if index_name not in pc.list_indexes():
#     print(f"Index '{index_name}' not found. Creating it...")
#     pc.create_index(
#         name=index_name,
#         dimension=384,  # Ensure this matches your embedding model
#         metric="cosine",
#         spec={ "cloud": "aws", "region": region}
#     )

# Connect to the existing index
index = pc.Index(index_name)
print(f"Connected to index: {index_name}")


Connected to index: medicalchatbot


In [89]:
import fitz  # PyMuPDF for PDF text extraction
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Pinecone, ServerlessSpec
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PineconeVectorStore

# 🔹 Step 1: Extract Text from Uploaded PDF
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file."""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# Path to the uploaded PDF
pdf_path = r"C:\Users\mohda\OneDrive\Desktop\MEDICAL-CHATBOT\data\Medical_book.pdf"  # Replace with the actual path

# Extract text from PDF
extracted_text = extract_text_from_pdf(pdf_path)

# 🔹 Step 2: Split Extracted Text into Smaller Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
text_chunks = text_splitter.split_text(extracted_text)

print(f"✅ Extracted {len(text_chunks)} text chunks from PDF.")

# 🔹 Step 3: Initialize Pinecone Client
pc = Pinecone(api_key="pcsk_5ftJtk_RPGwskkYfaFGx21SBduyqwS44EGQ2stdCX3rJRdhB1s32dos2WfAJMH6hRDhq3B")  # Replace with your actual API Key

# Define Pinecone index details
index_name = "medicalchatbot"
expected_dim = 384  # Ensure embeddings match this dimension



# Connect to the existing index
index = pc.Index(index_name)
print(f"Connected to index: {index_name}")

# 🔹 Step 4: Convert Text Chunks into Vector Embeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
embeddings = embedding_model.embed_documents(text_chunks)

# 🔹 Step 5: Format Data for Pinecone Upsert
vectors = [
    {"id": f"doc_{i}", "values": embeddings[i], "metadata": {"text": text_chunks[i]}}
    for i in range(len(text_chunks))
]

# 🔹 Step 6: Store Vectors in Pinecone
batch_size = 100  # To prevent exceeding Pinecone's request limits
for i in range(0, len(vectors), batch_size):
    index.upsert(vectors=vectors[i : i + batch_size])
    print(f"Upserted {i + len(vectors[i : i + batch_size])} vectors so far.")

print("✅ PDF text chunks successfully converted to vectors and stored in Pinecone!")


✅ Extracted 5896 text chunks from PDF.
Connected to index: medicalchatbot
Upserted 100 vectors so far.
Upserted 200 vectors so far.
Upserted 300 vectors so far.
Upserted 400 vectors so far.
Upserted 500 vectors so far.
Upserted 600 vectors so far.
Upserted 700 vectors so far.
Upserted 800 vectors so far.
Upserted 900 vectors so far.
Upserted 1000 vectors so far.
Upserted 1100 vectors so far.
Upserted 1200 vectors so far.
Upserted 1300 vectors so far.
Upserted 1400 vectors so far.
Upserted 1500 vectors so far.
Upserted 1600 vectors so far.
Upserted 1700 vectors so far.
Upserted 1800 vectors so far.
Upserted 1900 vectors so far.
Upserted 2000 vectors so far.
Upserted 2100 vectors so far.
Upserted 2200 vectors so far.
Upserted 2300 vectors so far.
Upserted 2400 vectors so far.
Upserted 2500 vectors so far.
Upserted 2600 vectors so far.
Upserted 2700 vectors so far.
Upserted 2800 vectors so far.
Upserted 2900 vectors so far.
Upserted 3000 vectors so far.
Upserted 3100 vectors so far.
Upser

In [97]:
from langchain.vectorstores import Pinecone as PineconeVectorStore
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from pinecone import Pinecone

# 🔹 Initialize Pinecone Client
pc = Pinecone(api_key="pcsk_5ftJtk_RPGwskkYfaFGx21SBduyqwS44EGQ2stdCX3rJRdhB1s32dos2WfAJMH6hRDhq3B")  # Replace with your API Key





# 🔹 Step 2: Define Index Name
index_name = "medicalchatbot"

# 🔹 Step 3: Get the Host of the Index
index_info = pc.describe_index(index_name)
host_url = "https://medicalchatbot-kvk4kva.svc.aped-4627-b74a.pinecone.io"  # Extract host URL

# 🔹 Step 4: Connect to Pinecone Index with Host
index = pc.Index(index_name, host=host_url)

print(f"✅ Connected to Pinecone index: {index_name}")

index_name = "medicalchatbot"


# 🔹 Initialize Embedding Model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 🔹 Connect to Existing Pinecone Index

# 🔹 Define Query
query = "what are headaches?"



query_result = index.query(
    vector=query_embedding,
    top_k=5,  # More results
    include_metadata=True
)

print("\n🔍 Query Results:")
for match in query_result["matches"]:
    score = match["score"]
    text = match["metadata"].get("text", "⚠ No matching text found")
    print(f"🔹 Confidence Score: {score:.4f}")
    print(f"📄 Retrieved Text: {text}\n")





✅ Connected to Pinecone index: medicalchatbot

🔍 Query Results:
🔹 Confidence Score: 0.5543
📄 Retrieved Text: ⚠ No matching text found

🔹 Confidence Score: 0.5457
📄 Retrieved Text: ⚠ No matching text found

🔹 Confidence Score: 0.5374
📄 Retrieved Text: heart are attached to the chest and extremities. The infor-
mation is displayed on a monitor screen or a paper tape
in the form of waves. Reduced blood and oxygen supply
to the heart shows as a change in the shape of the waves.
• Noninvasive vascular tests. These involve measuring
blood pressure in various parts of the body and compar-
ing the results from each location. When there is a
KEY TERMS
Arterial fibrillation—An arrhythmia; chaotic quiv-
ering of the arteries.

🔹 Confidence Score: 0.4930
📄 Retrieved Text: make the heart beat properly can be implanted under the
skin during a simple operation. Leads from the pacemak-
er are anchored to the right side of the heart. Pacemakers
are used to correct bradycardia and are sometimes used
aft

In [98]:
# Describe your Pinecone index
index_stats = index.describe_index_stats()
print("Index Stats:", index_stats)


Index Stats: {'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 11096},
                'medical_book_text': {'vector_count': 2}},
 'total_vector_count': 11098}


In [99]:
query_result = index.query(
    vector=[0] * 384,  # Dummy query vector
    top_k=5,
    include_metadata=True
)

# Print stored text chunks
print("\n🔍 Stored Data in Pinecone:")
for match in query_result["matches"]:
    print(f"📄 Text: {match['metadata'].get('text', 'No text found')}")



🔍 Stored Data in Pinecone:
📄 Text: No text found
📄 Text: nonprescription (over-the-counter) drugs without first
checking with a physician.
Because the atypical antipsychotics may cause low-
ering of blood pressure, care should be used when these
drugs are taken at the same time as other drugs which
lower blood pressure.
Quetiapine has many interactions. Doses should be
carefully adjusted when quetiapine is used with keto-
conazole, itraconazole, fluconazole, erythromycin, car-
bamazepine, barbiturates, rifampin or glucocorticoids
📄 Text: Publishers, May 1996.
PERIODICALS
McDougle, C. J. “A double-blind, placebo-controlled study of
risperidone addition in serotonin reuptake inhibitor-refrac-
tory obsessive-compulsive disorder.” Archives of General
Psychiatry (August 2000): 794.
Samuel David Uretsky, PharmD
Antiretroviral drugs
Definition
Antiretroviral drugs inhibit the reproduction of
retroviruses—viruses composed of RNA rather than
DNA. The best known of this group is HIV, human
📄 Te

In [100]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Load the same embedding model used for storage
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Convert query into vector
query_text = "What are allergies?"
query_embedding = embedding_model.embed_query(query_text)


In [101]:
query_result = index.query(
    vector=query_embedding, 
    top_k=5, 
    include_metadata=True
)

print("\n🔍 Query Results:")
for match in query_result["matches"]:
    score = match["score"]
    text = match["metadata"].get("text", "⚠ No matching text found")
    print(f"🔹 Confidence Score: {score:.4f}")
    print(f"📄 Retrieved Text: {text}\n")



🔍 Query Results:
🔹 Confidence Score: 0.7797
📄 Retrieved Text: GALE ENCYCLOPEDIA OF MEDICINE 2
118
Allergies

🔹 Confidence Score: 0.7743
📄 Retrieved Text: GALE ENCYCLOPEDIA OF MEDICINE 2
121
Allergies

🔹 Confidence Score: 0.7132
📄 Retrieved Text: ⚠ No matching text found

🔹 Confidence Score: 0.7030
📄 Retrieved Text: ⚠ No matching text found

🔹 Confidence Score: 0.6950
📄 Retrieved Text: ders Co., 1993.
Lawlor, G. J. Jr., T. J. Fischer, and D. C. Adelman. Manual of
Allergy and Immunology. Boston: Little, Brown and Co.,
1995.
Novick, N. L. You Can Do Something About Your Allergies.
New York: Macmillan, 1994.
Weil, A. Natural Health, Natural Medicine: A Comprehensive
Manual for Wellness and Self-Care. New York: Houghton
Mifflin, 1995.
Richard Robinson
Allergies
Definition
Allergies are abnormal reactions of the immune sys-
tem that occur in response to otherwise harmless sub-



In [102]:
query_text = "What are heart attacks?"
query_embedding = embedding_model.embed_query(query_text)

In [103]:
query_result = index.query(
    vector=query_embedding, 
    top_k=5, 
    include_metadata=True
)

print("\n🔍 Query Results:")
for match in query_result["matches"]:
    score = match["score"]
    text = match["metadata"].get("text", "⚠ No matching text found")
    print(f"🔹 Confidence Score: {score:.4f}")
    print(f"📄 Retrieved Text: {text}\n")



🔍 Query Results:
🔹 Confidence Score: 0.6189
📄 Retrieved Text: some patients after a heart attack. Heart attacks dam-
age and weaken the heart muscle, and the damage con-
tinues even after a person recovers from the attack. This
medicine helps slow down further damage to the heart.
ACE inhibitors also may be used to treat congestive
heart failure.
Description
ACE inhibitors are available only with a physician’s
prescription and come in tablet, capsule, and injectable
forms. Some commonly used ACE inhibitors are

🔹 Confidence Score: 0.6042
📄 Retrieved Text: ⚠ No matching text found

🔹 Confidence Score: 0.5980
📄 Retrieved Text: infarction (heart attack). It also may be done on some-
one with heart disease, such as cardiomyopathy or
rheumatic fever.
Although the presence of antimyocardial antibodies
does not diagnose heart damage or disease, there is a
connection between the presence of these antibodies and
damage to the heart. The amount of damage, however,
cannot be predicted by the amo